# Project: Study of Electrodes' Ability, Attached to Tissue to Distinct Nonlinear Behaviour of the Cell

> Course: Bioelectrical Phenomena

> Supervisor: Dr. Mehrdad Saviz

> Teaching Assistant: Eng. Tabibi 

> Authors: Mahya Ahandoust 40033010- Mohmmad Amin Heydarian 40033018- Mahtab Dastranj 40033024

> Date: January 2025

## A. Problem

In this problem, we want to investigate the ability to detect non-linear behavior of a cell through electrodes attached to the tissue.
Our main method is to calculate the second and third harmonics of the current generated by the membrane and calculate the voltage value that can be recorded by the electrode due to this current activity on the membrane. To solve the problem accurately, a programmer will be needed. But check the approximate numbers obtained with manual calculations to prevent errors.

### 1. Cell Geometry:
Consider the cell as spherical with a diameter of 10 micrometers, approximated by a cube with the same dimensions. Assume that the cell contains Cav2.1-type channels.


### 2. Electrode Configuration:
Assume this cell is located within a tissue environment and connected via two electrodes positioned 1 cm apart. An electric field with an amplitude of E=500V/m and a frequency of f=5Hz is applied. These electrodes are also used to record the electrical activity induced in the cell.

### 3. Membrane Potential and Ionic Currents:
Compute the induced voltage on the membrane, assuming the membrane has low conductivity. Next, calculate the ionic currents through the ion channels. Use the existing model in Channelpedia to determine the conductivity of the channels and assume reasonable values for all quantities, such as the number of channels per unit membrane area, medium conductivity, resting potential, etc., based on the course material.

### 4. Dipole Representation:
Consider a small piece of the membrane as a dipole with a known current. Calculate the voltage recorded by the electrodes from this dipole.

### 5. Harmonics of Recorded Voltage:
Determine the second and third harmonics of the recorded voltage.

### 6. Detectability of Electrical Activity:
If the harmonics are at least 
10μV, they are considered detectable by the digital recording system. Analyze the conditions under which this is possible using the following parameters:
a. Applied field intensity.
b. Number of cells in the same area with similar properties.

### Simplifications:

- For ease of analysis, investigate whether the channel behavior can be approximated under low-frequency conditions.

- Assume that sustained cell stimulation does not result in changes to concentrations or equilibrium voltage within the cell.

## B. Manual Calculations

**Cell Geometric**

$V_{cell} = \frac{4}{3} \pi \left( \frac{\text{diameter}}{2} \right)^3 = \frac{4}{3}\pi(5\times10^{-6}m)^{3}=5.236\times10^{-16}m^{3}$

- Cube side length equivalent: Since we approximate the cell as a cube with the same volume, the side length of the cube is:

$Side~length~of~cube = V_{cell} ^ {\frac{1}{3}} = (5.236\times10^{-16})^{\frac{1}{3}}$

- Surface area of the membrane (cube approximation):
$$A_{membrane} = 6\times(side  length)^2 = 6\times(8.06\times10^{-6})^2 = 3.897\times10^{-10} m^2$$